In [1]:
%load_ext autoreload
%autoreload 2

Using simpler models with the same results. Can this be considered a victory?

In [2]:
import sys
import os
import pandas as pd
import shap
import csv
sys.path.append(os.path.abspath(os.path.join('..')))
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, ParameterSampler
from imblearn.over_sampling import RandomOverSampler, SMOTE
from sklearn.metrics import classification_report

from utility.classification_utility import *

c:\Users\Yuri\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Let's start by building our beautiful dataset

In [3]:
cyc = '../dataset/cyclists_cleaned.csv'
races = '../dataset/races_cleaned.csv'
df = make_dataset_for_classification(races, cyc, make_stage_type=True, make_race_participants=True)

100.00%  


In [4]:
# dumb imputations
df['height'] = df['height'].fillna(df['height'].mean())
df['weight'] = df['weight'].fillna(df['weight'].mean())
df['bmi'] = df['weight']/np.square(df['height']/100)
df['cyclist_age_rac'] = df['cyclist_age_rac'].fillna(df['cyclist_age_rac'].mean())
df['climb_total'] = df['climb_total'].fillna(df['length']*0.05) # could be differentiated by lengths
df['steepness'] = df['length'] / df['climb_total']

In [5]:
# for profile, let's use the big stick
df['is_ITT'] = (df['stage_type'] == 'ITT').astype(int)
df_filtered = df[['length', 'climb_total', 'steepness', 'average_speed', 'is_ITT', 'profile']].dropna()
X = df_filtered[['length', 'climb_total', 'steepness', 'average_speed', 'is_ITT']]
y = df_filtered['profile']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [6]:
y_pred = clf.predict(X_test)
print(f"Accuracy: {np.mean(y_pred == y_test)}")
print(f"Feature importance: {clf.feature_importances_}")

Accuracy: 0.9991312777770783
Feature importance: [0.21644605 0.27903885 0.30263699 0.20187812 0.        ]


Well.. this might mean that the inferences over the climb total and the length were quite accurate

In [7]:
# Predict missing profiles
missing_profiles = df[df['profile'].isna()]
df.loc[missing_profiles.index, 'profile'] = clf.predict(missing_profiles[['length', 'climb_total', 'steepness', 'average_speed', 'is_ITT']])

In [8]:
TO_USE_COLS = [
    # over time
    'total_points',
    'avg_points_per_race', 
    'average_position',
    'avg_speed_cyclist', 
    'mean_stamina_index',
    'race_count',
    # race related
    'length',
    'climb_total', 
    'profile', 
    'startlist_quality', 
    'cyclist_age_rac', 
    'steepness', 
    'is_tarmac', 
    'stage_type',
    'season',
    'total_participants',
    # cyclist related
    'height',
    'weight',
    'bmi',
    'home_game',
    # for the split
    'date',
    'target'
]

df_to_use = df[TO_USE_COLS]

In [9]:
# useless
# season_dummies = pd.get_dummies(df['season'], prefix='season')
# df_to_use = pd.concat([df_to_use.drop(columns=['season']), season_dummies], axis=1)

one_hot encoding season gave no sensible results. As expected, the feature remains quite uninsightful

In [9]:
df_tr, df_vl, df_ts = get_data_split(df_to_use)

df_tr = df_tr.drop(columns=['date'])
df_vl = df_vl.drop(columns=['date'])
df_ts = df_ts.drop(columns=['date'])

X_tr, y_tr = split_features_target(df_tr)
X_vl, y_vl = split_features_target(df_vl)
X_ts, y_ts = split_features_target(df_ts)

Normalize the features before predicting

In [10]:
scal = StandardScaler()
X_tr = scal.fit_transform(X_tr)
X_vl = scal.transform(X_vl)
X_ts = scal.transform(X_ts)

# Nn test

This is a funny tool for later

In [11]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_recall',  
    patience=5,            
    mode='max',            
    restore_best_weights=True  
)

Let's build this model

In [18]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(X_tr.shape[1],)),
    tf.keras.layers.Dense(64, activation='tanh'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='tanh'),
    tf.keras.layers.Dropout(0.5),
    # tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

optimizer = tf.keras.optimizers.Lion()


model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=[
    'accuracy',
    'recall'
    ])

Now fit it

In [19]:
history = model.fit(
    X_tr, y_tr, 
    validation_data=(X_vl, y_vl), 
    epochs=10, 
    batch_size=64,
    verbose=2,
    callbacks=[early_stopping]
)

Epoch 1/10
5550/5550 - 3s - 589us/step - accuracy: 0.8514 - loss: 0.3793 - recall: 0.0743 - val_accuracy: 0.8567 - val_loss: 0.3531 - val_recall: 0.0114
Epoch 2/10
5550/5550 - 3s - 489us/step - accuracy: 0.8547 - loss: 0.3728 - recall: 0.0716 - val_accuracy: 0.8588 - val_loss: 0.3495 - val_recall: 0.0436
Epoch 3/10
5550/5550 - 3s - 493us/step - accuracy: 0.8548 - loss: 0.3715 - recall: 0.0744 - val_accuracy: 0.8581 - val_loss: 0.3512 - val_recall: 0.0556
Epoch 4/10
5550/5550 - 3s - 494us/step - accuracy: 0.8547 - loss: 0.3716 - recall: 0.0757 - val_accuracy: 0.8578 - val_loss: 0.3514 - val_recall: 0.0462
Epoch 5/10
5550/5550 - 3s - 495us/step - accuracy: 0.8549 - loss: 0.3720 - recall: 0.0764 - val_accuracy: 0.8601 - val_loss: 0.3512 - val_recall: 0.0877


In [20]:
val_pred = model.predict(X_vl)
val_pred = (val_pred > 0.5).astype(int)
report = classification_report(y_vl, val_pred)
print(report)

1359/1359 ━━━━━━━━━━━━━━━━━━━━ 0s 306us/step
              precision    recall  f1-score   support

       False       0.86      1.00      0.92     37243
        True       0.52      0.01      0.02      6237

    accuracy                           0.86     43480
   macro avg       0.69      0.50      0.47     43480
weighted avg       0.81      0.86      0.79     43480



# Grid Search

In [12]:
dict_learning = {
    'adam': tf.keras.optimizers.Adam(),
    'sgd': tf.keras.optimizers.SGD(),
    'lion': tf.keras.optimizers.Lion()
}

def build_model(num_layers=2, num_units=64, optimizer='adam', activation='relu', dropout=0.2):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Input(shape=(X_tr.shape[1],)))
    
    model.add(tf.keras.layers.Dense(num_units, activation=activation))
    model.add(tf.keras.layers.Dropout(dropout))
    
    for _ in range(num_layers - 1):
        model.add(tf.keras.layers.Dense(num_units, activation=activation))
        model.add(tf.keras.layers.Dropout(dropout))
    
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    
    opt = dict_learning[optimizer]
    
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy', 'recall'])
    return model

param_grid = {
    'num_layers': [1, 2],
    'num_units': [32, 64],
    'optimizer': ['adam', 'sgd', 'lion'],
    'activation': ['relu', 'tanh'],
    'batch_size': [16, 32, 64],
    'epochs': [5],
} # already passed through this

This proto grid search has already been ran over various parameters. Without verbosity. Won't be uploading on git a long version of the last.

Nothing much changes between these different hyperparameters. Maybe the answer lies elsewhere, like in regularization: oversampling and dropout?

In [14]:
param_grid = {
    'dropout': [0.2, 0.5],
    'num_layers': [1, 2, 3],
    'num_units': [32, 64, 128],
    'optimizer': ['adam', 'sgd', 'lion'],
    'activation': ['relu', 'tanh'],
    'batch_size': [16,32,64],
    # 'batch_norm': [True], # useless
    'epochs': [5,10],
}

In [15]:
ratio = 0.35
# oversample = RandomOverSampler(sampling_strategy=ratio, random_state=42) # same results as SMOTE
oversample = SMOTE(sampling_strategy=ratio, random_state=42)
oversampled_X, oversampled_y = oversample.fit_resample(X_tr, y_tr)

Over the different trials, SMOTE gave no sensible difference from usual random oversampling.

In [ ]:
param_list = list(ParameterSampler(param_grid, n_iter=650, random_state=42))

for params in param_list:
    print(f"Training model with parameters: {params}")
    model = build_model(params['num_layers'], params['num_units'], params['optimizer'], params['activation'], params['dropout'])
    history = model.fit(
        oversampled_X, oversampled_y, 
        validation_data=(X_vl, y_vl), 
        epochs=params['epochs'], 
        batch_size=params['batch_size'],
        verbose=0,
        callbacks=[early_stopping]
    )
    val_pred = model.predict(X_vl)
    val_pred = (val_pred > 0.5).astype(int)
    
    # write metrics and params in a file
    with open('model_eval.txt', 'a') as f:
        f.write(f"Parameters: {params}, oversampled: {ratio}\n")
        f.write(classification_report(y_vl, val_pred))
        f.write('\n\n')


In [ ]:
param_list = list(ParameterSampler(param_grid, n_iter=650, random_state=42))

# Initialize CSV file if not exists
csv_file = 'model_eval.csv'
write_header = not os.path.exists(csv_file)

# Loop through each parameter set and train the model
for params in param_list:
    print(f"Training model with parameters: {params}")
    model = build_model(params['num_layers'], params['num_units'], params['optimizer'], params['activation'], params['dropout'])
    history = model.fit(
        oversampled_X, oversampled_y, 
        validation_data=(X_vl, y_vl), 
        epochs=params['epochs'], 
        batch_size=params['batch_size'],
        verbose=0,
        callbacks=[early_stopping]
    )
    val_pred = model.predict(X_vl)
    val_pred = (val_pred > 0.5).astype(int)
    
    # Generate classification report as a DataFrame
    report = classification_report(y_vl, val_pred, output_dict=True)
    report_df = pd.DataFrame(report).transpose()
    
    # Add model parameters to the DataFrame
    for param, value in params.items():
        report_df[param] = value
    report_df['oversampled_ratio'] = ratio
    
    # Append DataFrame to the CSV file
    if write_header:
        report_df.to_csv(csv_file, mode='w', index_label='class')
        write_header = False  # Write header only for the first iteration
    else:
        report_df.to_csv(csv_file, mode='a', header=False, index_label='class')


Oversampling boosts recall to good levels but mutilates precision, while dropout does the opposite. Tuning them seems the best option <br>
Two layers and more units seem to improve a bit the overall metrics. <br>
As professor Micheli said, I have to listen to the neural network and move where it tells me.  <br>
Also, early stopping and more epochs might help. Maybe let's reduce the parameters.

In [25]:
# get dataset from csv in model_eval.csv
report = pd.read_csv('model_eval.csv')
# keep only the macro average
report = report[report['class'] == 'macro avg']

In [26]:
report.describe()

,precision,recall,f1-score,support,num_units,num_layers,epochs,dropout,batch_size,oversampled_ratio
count,648.000000,648.000000,648.000000,648.0,648.000000,648.000000,648.000000,648.000000,648.000000,6.480000e+02
mean,0.668154,0.631426,0.638729,43480.0,74.666667,2.000000,7.500000,0.350000,37.333333,3.500000e-01
std,0.052247,0.042353,0.048476,0.0,39.941843,0.817127,2.501931,0.150116,19.970922,5.555403e-17
min,0.428271,0.499946,0.461341,43480.0,32.000000,1.000000,5.000000,0.200000,16.000000,3.500000e-01
25%,0.657668,0.620079,0.635100,43480.0,32.000000,1.000000,5.000000,0.200000,16.000000,3.500000e-01
50%,0.677673,0.642081,0.651202,43480.0,64.000000,2.000000,7.500000,0.350000,32.000000,3.500000e-01
75%,0.690570,0.659366,0.666795,43480.0,128.000000,3.000000,10.000000,0.500000,64.000000,3.500000e-01
max,0.846577,0.691563,0.680369,43480.0,128.000000,3.000000,10.000000,0.500000,64.000000,3.500000e-01


In [29]:
report.sort_values(by='f1-score', ascending=False)

,class,precision,recall,f1-score,support,optimizer,num_units,num_layers,epochs,dropout,batch_size,activation,oversampled_ratio
2913,macro avg,0.685736,0.675539,0.680369,43480.0,adam,128,2,10,0.2,64,tanh,0.35
123,macro avg,0.683576,0.677202,0.680283,43480.0,adam,128,3,5,0.2,16,relu,0.35
1113,macro avg,0.686514,0.674686,0.680241,43480.0,adam,128,1,5,0.2,64,relu,0.35
663,macro avg,0.686139,0.674872,0.680179,43480.0,adam,128,3,5,0.2,32,relu,0.35
228,macro avg,0.673548,0.687022,0.679785,43480.0,adam,32,3,10,0.2,16,relu,0.35
...,...,...,...,...,...,...,...,...,...,...,...,...,...
523,macro avg,0.428277,0.500000,0.461368,43480.0,lion,64,3,10,0.5,16,relu,0.35
133,macro avg,0.428277,0.500000,0.461368,43480.0,lion,128,3,5,0.2,16,relu,0.35
808,macro avg,0.428277,0.500000,0.461368,43480.0,lion,128,3,10,0.2,32,relu,0.35
118,macro avg,0.428277,0.500000,0.461368,43480.0,lion,64,3,5,0.2,16,relu,0.35


All in all, it seems to be mostly about initialization and not falling in bad local minima. A good balance of dropout and oversampling managed to get us up to `0.68` of f1-score. Maybe, with better features, we could have achieved more.

# Explanability

Let's see what can we infer from applying some explanability to these models! So far we don't move past 0.68, no matter what we do. Maybe shap can give us some insights

In [ ]:
shap.initjs()

In [ ]:
# build a winning model
winnable = build_model()
winnable.fit(oversampled_X, oversampled_y, 
        validation_data=(X_vl, y_vl), 
        epochs=10, 
        batch_size=64,
        verbose=1,
        callbacks=[early_stopping])

In [17]:
# make X_vl into a DataFrame
X_df_vl = pd.DataFrame(X_vl, columns=df_tr.columns[:20])

In [39]:
sample_to_train = shap.sample(X_df_vl, 50)

In [50]:
def f(X):
    return winnable.predict(X).flatten()

In [ ]:
# explain a bunch of predictions using shap

explainer = shap.KernelExplainer(f, sample_to_train.iloc[:50])

In [ ]:
shap_values = explainer.shap_values(X_df_vl.iloc[:100,:], nsamples=400)

In [ ]:
shap.force_plot(explainer.expected_value, shap_values, X_df_vl.iloc[:100,:])

In [ ]:
shap.summary_plot(shap_values, X_df_vl.iloc[:100,:])

As can be seen from the force plot and the summary plot, we have something to talk about!

`average_position`, `cyclist_age_rac` and `profile`:
- These features have the most significant impact on the model's output, with SHAP values distributed widely.
- Different values of these features strongly influence predictions, as indicated in the plot.
- As we might have expected, statistics that might give an insight on the preceding positions of the cyclists are used a lot by our algorithm. Maybe, the standard deviation of it and other transformations might have given some more edge to our models
- As it seems, an higher age (at race time!) implies less chances to win a race. Intuitively, this can be seen as cyclists having more chances to get good positions when they are in their "prime" instead of when they hit the 50 years of age.
- It seems the profile of a race highly influences whether a cyclist will arrive on the podium or not! This might mean that there are a lot of cyclists that are more inclined to win certain races than others.

`avg_points_per_race` and `total_points`:
- These features also play a critical role but have slightly less variance in their SHAP values compared to the top features. These general statistics about how fructuous a cyclist performances are help the model infer its possible position more than the mere race count.

`avg_speed_cyclist`:
- This feature seems moderately important, with a noticeable spread around the center for very high values and very skewed values for the lowest values. It's difficult to get to the finish line first if on average you are slow.

`race_count`, `startlist_quality`, `mean_stamina_index` and `weight`:
- These have limited impact compared to other features but still contribute marginally. It seems that, for this model, heavier cyclists have a harder time winning.

Minor Features:
- Features such as `bmi`, `height`, `home_game`, and `steepness` have a minimal impact on the model, as indicated by their tightly clustered SHAP values around zero.